In [20]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,roc_auc_score,ConfusionMatrixDisplay,precision_score,recall_score


In [3]:
import numpy as np
from scipy import signal

def preprocess(data):
  # Delete silent part
  data = data.astype(float)
  n_size = 50
  n_len = int(data.shape[0]/n_size)
  std_data = np.zeros((n_size, 2))
  for i in range(n_size):
    seg_data_x = data[i*n_len:i*n_len+n_len, 0]
    seg_data_y = data[i*n_len:i*n_len+n_len, 1]
    std_data[i, 0] = np.std(seg_data_x)
    std_data[i, 1] = np.std(seg_data_y)
  pass_threshold = 1
  pass_idx_x = np.where(std_data[:,0] >= pass_threshold)[0]
  pass_idx_y = np.where(std_data[:,1] >= pass_threshold)[0]
  if len(pass_idx_x) == 0:
    start_idx = max(0, pass_idx_y[0] - 1)
    end_idx = min(data.shape[0],pass_idx_y[-1] + 1)
  elif len(pass_idx_y) == 0:
    start_idx = max(0, pass_idx_x[0] - 1)
    end_idx = min(data.shape[0],pass_idx_x[-1] + 1)
  else:
    start_idx = max(0,min(pass_idx_x[0], pass_idx_y[0]) - 1)
    end_idx = min(data.shape[0],max(pass_idx_x[-1], pass_idx_y[-1]) + 1)
  
  # resample to 100 data points
  data = signal.resample(data[start_idx*n_len:end_idx*n_len, :], 100, axis=0)
  # scale
  data = (data - data.min(axis=0, keepdims=True))/(data.max(axis=0, keepdims=True) - data.min(axis=0, keepdims=True))
  return data


In [4]:

drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
main_dir = 'drive/MyDrive/2023JHU/MLMA_EOG/isolated 2/'
os.listdir(main_dir)

['ReadMe', '.DS_Store', '005', '004', '003', '001', '006', '002']

In [6]:
# get file list for 1 patient
def get_file_list(path):
    file_list = []
    label = []
    for i in os.listdir(path):
        file_list.append(i)
        l = int(i.split("_")[-1].split(".")[0])-1
        label.append(l)
    return label,file_list

In [16]:
# get feature from 1 file and preprocess
def get_feature(path):
    f = []
    a = pd.read_csv(path,names=["vertical","horizontal"])
    a = np.array(a)
    #print(a.shape)
    a = preprocess(a)
    #print(a.shape)
    f.append(a)        

    return f
    
    

In [8]:
# self identified test_split
def my_train_test_split_user_dependent(path,test_split,file_list,label):
    X_test = []
    X_train = []
    y_train = []
    y_test = []
    for f in range(len(file_list)):
        file = file_list[f]
        file_label = label[f]
        feature = get_feature(str(path+file))
        #print(file.split('_')[2],file_label)
        if file.split('_')[2] in test_split:
            X_test.append(feature)
            y_test.append(file_label)
            #print(file,len(feature))
        else:
            X_train.append(feature)
            y_train.append(file_label)
            #print(file)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return X_train,X_test, y_train,y_test

In [9]:
def evaluate(y_true, y_pred,y_prob):
    f1_micro = f1_score(y_true, y_pred,average = 'micro')
    f1_macro = f1_score(y_true, y_pred,average = 'macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)

    return f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc

In [32]:
class classficiation(nn.Module):
    def __init__(self):
        super(classficiation, self).__init__()
        self.encoder = nn.Sequential(
            # your codes here
            nn.Conv2d(1,32,2,stride = 1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32,momentum=0.99),
            nn.Dropout(p=0.5),
            nn.Conv2d(32,16,1,stride = 1,padding=1),
            nn.ReLU()   
        )


        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8240,200),
            nn.ReLU(),
            nn.Linear(200,12)
            #nn.Softmax()
        )



    def forward(self, x):
         # your codes here
        #print(x.shape)
        encoder = self.encoder(x)
        #print(encoder.shape)
        x = self.classifier(encoder)

        return x 




In [11]:
# Define custom dataset class
class MYDataset(Dataset):
    def __init__(self, feature,label):
      self.feature = []
      self.label = []
      for i in range(len(feature)):
        self.feature.append(feature[i])
        self.label.append(label[i])

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        f = self.feature[idx]
        #f = f[np.newaxis,:,:]
        l = self.label[idx]
        return f, l


# User Dependent

In [12]:
patient = ["001","002","003","004","005","006"]
test_split = [["01","02"],["03","04"],["05","06"],["07","08"],["09","10"]]

In [36]:
#### def training
def train(model,train_dataloader,num_epochs,optimizer,criterion):
  train_loss = []
  model = model.to(device)
  for epoch in range(num_epochs):
        #print("running epoch: ", epoch)
        model.train()
        Loss = 0
        train_label = []
        train_ypred = []
        for i, (feature,label) in enumerate(train_dataloader, 0):
          #print(feature.shape)
          optimizer.zero_grad()
          feature = feature.to(device)
          output = model(feature.float())
          ypred = torch.argmax(output.cpu(),dim=1)
          #print(feature.shape,output.shape)
          #print(ypred)
          for j in ypred:
            train_ypred.append(j)
          for j in label:
            train_label.append(j)
          loss = criterion(output,label.to(device))
          loss.backward()
          optimizer.step()
          Loss += loss.item()
        Loss = Loss/(i+1)
        acc = accuracy_score(y_true=train_label, y_pred= train_ypred)
        train_loss.append(Loss)
  #model.eval()

        #print("train loss: ",Loss," train accuracy: ",acc)
  return model

In [22]:
def eval(model,test_dataloader,m):
  with torch.no_grad():
    model.eval()
    label_list = []
    ypred_list = []
    yprob_list = []
        #Loss = 0
    for i, (feature,label) in enumerate(test_dataloader, 0):
      feature = feature.to(device)
      output = model(feature.float())
      #print(output.shape)
      ypred = torch.argmax(output,dim=1)
      yprob = m(output)
      for j in ypred.cpu().detach().numpy():
        ypred_list.append(j)
      for l in label.cpu().detach().numpy():
        label_list.append(l)
      for l in yprob.cpu().detach().numpy():
        yprob_list.append(l)
    f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc = evaluate(label_list, ypred_list,yprob_list)

    #print("test acc: ", acc)
    return f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc

## train 5-fold cross validation

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
m = nn.Softmax(dim=1).to(device)
total_acc = []
total_f1_micro = []
total_f1_macro = []
total_recall_micro = []
total_recall_macro = []
total_precision_micro = []
total_precision_macro = []
for p in patient:
    #print(p)
    output_pck = []
    path = str(main_dir+p+"/isolated_strokes/")
    acc = 0 
    f1_macro =0
    f1_micro =0
    recall_micro =0
    recall_macro =0
    precision_micro =0
    precision_macro =0
    for t in test_split:
        label,file_list = get_file_list(path)
        X_train,X_test, y_train,y_test = my_train_test_split_user_dependent(path,t,file_list,label)
        #print(X_train.shape,X_test.shape, y_train.shape,y_test.shape)
        train_data = MYDataset(X_train,y_train)
        train_dataloader  = DataLoader(train_data, batch_size=16, shuffle=True)
        test_data = MYDataset(X_test,y_test)
        test_dataloader  = DataLoader(test_data, batch_size=16, shuffle=True)
        
        # set new model
        torch.manual_seed(42)
        model = classficiation()
        lr = 0.0001
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        criterion = nn.CrossEntropyLoss()
        num_epochs = 50
        # train 
        model = train(model,train_dataloader,num_epochs,optimizer,criterion)
        # evaluate 
        f1_mi,f1_ma,precision_mi,precision_ma,recall_mi,recall_ma,accuracy = eval(model,test_dataloader,m)
        acc += accuracy
        f1_macro += f1_ma
        f1_micro += f1_mi
        recall_micro += recall_mi
        recall_macro += recall_ma
        precision_micro += precision_mi
        precision_macro += precision_ma
    
    print(p,acc/5,f1_macro/5,f1_micro/5,recall_micro/5,recall_macro/5,precision_micro/5,precision_macro/5)
    total_acc.append(acc/5)
    total_f1_micro.append(f1_micro/5)
    total_f1_macro.append(f1_macro/5)
    total_recall_micro.append(recall_micro/5)
    total_recall_macro.append(recall_macro/5)
    total_precision_micro.append(precision_micro/5)
    total_precision_macro.append(precision_macro/5)



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

001 0.7166666666666666 0.6617460317460317 0.7166666666666666 0.7166666666666666 0.7166666666666666 0.7166666666666666 0.6830555555555555


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

002 0.725 0.6712698412698413 0.725 0.725 0.725 0.725 0.6883333333333332


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


005 0.7148550724637681 0.6673809523809523 0.7148550724637681 0.7148550724637681 0.7166666666666666 0.7148550724637681 0.7022222222222222


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


006 0.7583333333333333 0.7279365079365079 0.7583333333333333 0.7583333333333333 0.7583333333333333 0.7583333333333333 0.7827777777777778


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
print("Total acc",np.mean(np.array(total_acc)))
print("f1_macro",np.mean(np.array(total_f1_macro)))
print("f1_micro",np.mean(np.array(total_f1_micro)))
print("total_recall_micro",np.mean(np.array(total_recall_micro)))
print("total_recall_macro",np.mean(np.array(total_recall_macro)))
print("total_precision_micro",np.mean(np.array(total_precision_micro)))
print("total_precision_macro",np.mean(np.array(total_precision_macro)))

Total acc 0.6802536231884057
f1_macro 0.6249867724867725
f1_micro 0.6802536231884057
total_recall_micro 0.6802536231884057
total_recall_macro 0.6805555555555554
total_precision_micro 0.6802536231884057
total_precision_macro 0.6499074074074075


In [47]:
#########training
for p in patient:
    print(p)
    path = str(main_dir+p+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    X = []
    y = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for f in range(len(file_list)):
        file = file_list[f]
        file_label = label[f]
        feature = get_feature(str(path+file))
        #print(file.split('_')[2],file_label)
        X.append(feature)
        y.append(file_label)
    X = np.array(X)
    y = np.array(y)
    train_data = MYDataset(X,y)
    train_dataloader  = DataLoader(train_data, batch_size=16, shuffle=True)
        # set new model
    torch.manual_seed(42)
    model = classficiation()
    lr = 0.0001
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    num_epochs = 50
        # train 
    model = train(model,train_dataloader,num_epochs,optimizer,criterion)
    state_dict = model.state_dict()
    torch.save(state_dict, str('drive/MyDrive/2023JHU/MLMA_EOG/checkpoint/CNN_sub'+str(p)+'.pth'))

001
002
003
004
005
006


# Split on patient level - User-independent

In [44]:
# self identified test_split
def my_train_test_split_user_independent(test_patient,train_patient):
    X_test = []
    X_train = []
    y_train = []
    y_test = []
    for p in train_patient:
        path = str(main_dir+p+"/isolated_strokes/")
        label,file_list = get_file_list(path)
        for i in range(len(file_list)):
            file = file_list[i]
            file_label = label[i]
            feature = get_feature(str(path+file))
            X_train.append(feature)
            y_train.append(file_label)
        
    path = str(main_dir+test_patient+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
        file = file_list[i]
        file_label = label[i]
        feature = get_feature(str(path+file))
        X_test.append(feature)
        y_test.append(file_label)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return X_train,X_test, y_train,y_test

In [46]:
patient = ["001","002","003","004","005","006"]
acc = 0 
f1_macro =0
f1_micro =0
recall_micro =0
recall_macro =0
precision_micro =0
precision_macro =0
for test_patient in patient:
    train_patient= ["001","002","003","004","005","006"]
    train_patient.remove(test_patient)
    X_train,X_test, y_train,y_test = my_train_test_split_user_independent(test_patient,train_patient)
    train_data = MYDataset(X_train,y_train)
    train_dataloader  = DataLoader(train_data, batch_size=16, shuffle=True)
    test_data = MYDataset(X_test,y_test)
    test_dataloader  = DataLoader(test_data, batch_size=16, shuffle=True)
        
    # set new model
    torch.manual_seed(42)
    model = classficiation()
    lr = 0.0001
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    num_epochs = 50
        # train 
    model = train(model,train_dataloader,num_epochs,optimizer,criterion)
        # evaluate 
    # evaluate 
    f1_mi,f1_ma,precision_mi,precision_ma,recall_mi,recall_ma,accuracy = eval(model,test_dataloader,m)
    acc += accuracy
    f1_macro += f1_ma
    f1_micro += f1_mi
    recall_micro += recall_mi
    recall_macro += recall_ma
    precision_micro += precision_mi
    precision_macro += precision_ma
    
print(acc/6,f1_macro/6,f1_micro/6,recall_micro/6,recall_macro/6,precision_micro/6,precision_macro/6)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


006 0.6964665281729365 0.670845012589444 0.6964665281729365 0.6964665281729365 0.6961560044893379 0.6964665281729365 0.728738795926296


In [48]:
#########training
X = []
y = []
for p in patient:
    print(p)
    path = str(main_dir+p+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for f in range(len(file_list)):
        file = file_list[f]
        file_label = label[f]
        feature = get_feature(str(path+file))
        #print(file.split('_')[2],file_label)
        X.append(feature)
        y.append(file_label)
X = np.array(X)
y = np.array(y)
train_data = MYDataset(X,y)
train_dataloader  = DataLoader(train_data, batch_size=16, shuffle=True)
        # set new model
torch.manual_seed(42)
model = classficiation()
lr = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 50
        # train 
model = train(model,train_dataloader,num_epochs,optimizer,criterion)
state_dict = model.state_dict()
torch.save(state_dict, 'drive/MyDrive/2023JHU/MLMA_EOG/checkpoint/CNN_subAll.pth')


001
002
003
004
005
006
